In [9]:
import pymongo
from datetime import timedelta, date
import charts

In [3]:
client = pymongo.MongoClient('localhost',27017)
ganji = client['ganji']
item_sample = ganji['item_sample']

In [24]:
#展示二手，二手笔记本，台式电脑：
#北京二手手机，北京而后搜台式机/配件，北京二手笔记本
for i in item_sample.find({'cates': {'$in':['北京二手手机','北京二手台式机/配件','北京二手笔记本']},'pub_date': {'$in': ['2015.12.31', '2016.01.14']}}).limit(300):
    print(i)
    

{'cates': ['北京58同城', '北京二手市场', '北京二手笔记本'], 'time': 2, '_id': ObjectId('5698f525a98063dbe8e91ca8'), 'price': 0, 'area': ['丰台'], 'look': '-', 'url': 'http://bj.58.com/bijibendiannao/24220974748090x.shtml', 'title': '【图】转让自用的电脑 保护的很新，几乎没用 - 丰台笔记本电脑 - 北京58同城', 'pub_date': '2015.12.31'}
{'cates': ['北京58同城', '北京二手市场', '北京二手笔记本'], 'time': 0, '_id': ObjectId('5698f525a98063dbe8e91ca9'), 'price': 0, 'area': ['朝阳', '国贸'], 'look': '-', 'url': 'http://bj.58.com/bijibendiannao/24516548452660x.shtml', 'title': '东芝 C600系列 笔记本 - 朝阳国贸笔记本电脑 - 北京58同城', 'pub_date': '2015.12.31'}
{'cates': ['北京58同城', '北京二手市场', '北京二手笔记本'], 'time': 3, '_id': ObjectId('5698f526a98063dbe8e91caa'), 'price': 1100, 'area': ['燕郊', '燕顺路'], 'look': '-', 'url': 'http://bj.58.com/bijibendiannao/24516186232254x.shtml', 'title': '【图】宏基 4750系列 笔记本 - 燕郊燕顺路笔记本电脑 - 北京58同城', 'pub_date': '2015.12.31'}
{'cates': ['北京58同城', '北京二手市场', '北京二手台式机/配件', '北京二手硬件', '北京二手电源'], 'time': 0, '_id': ObjectId('5698f527a98063dbe4e91ca9'), 'price': 50, 'area': 

In [7]:
for i in item_sample.find():
    frags = i['pub_date'].split('-')
    if len(frags) == 1:
        date = frags[0]
    else:
        date = '{}.{}.{}'.format(frags[0], frags[1], frags[2])
    item_sample.update_one({'_id':i['_id']}, {'$set':{'pub_date':date}})

In [10]:
def get_all_dates(date1, date2):
    the_date = date(int(date1.split('.')[0]), int(date1.split('.')[1]), int(date1.split('.')[2]))
    end_date = date(int(date2.split('.')[0]), int(date2.split('.')[1]), int(date2.split('.')[2]))
    days = timedelta(days=1)
   
    while the_date <= end_date:
        yield (the_date.strftime('%Y.%m.%d'))
        the_date = the_date + days

    

In [37]:
def get_data_with_in(date1, date2, cates):
    for cate in cates:
        cate_day_post = []
        for date in get_all_dates(date1, date2):
            a = list(item_sample.find({'pub_date': date, 'cates':cate}))
            each_day_post = len(a)
            cate_day_post.append(each_day_post)
        data = {
            'name': cate,
            'data': cate_day_post,
            'type':'line'
        }
        yield data

In [38]:
for i in get_data_with_in('2015.12.24', '2016.01.05', ['北京二手手机','北京二手台式机/配件','北京二手笔记本']):
    print(i)

{'type': 'line', 'data': [101, 91, 89, 68, 78, 81, 57, 81, 82, 105, 118, 103, 92], 'name': '北京二手手机'}
{'type': 'line', 'data': [62, 28, 56, 145, 186, 164, 151, 151, 142, 167, 197, 193, 206], 'name': '北京二手台式机/配件'}
{'type': 'line', 'data': [43, 60, 57, 53, 50, 48, 52, 53, 46, 53, 53, 84, 50], 'name': '北京二手笔记本'}


In [42]:
#进行数据可视化
options = {
    'chart'   : {'zoomType':'xy'},
    'title'   : {'text': '发帖量统计'},
    'subtitle': {'text': '可视化统计图表'},
    'xAxis'   : {'categories': [i for i in get_all_dates('2015.12.24', '2016.01.05')], 'title':{'text':'日期'}},
    'yAxis'   : {'title': {'text': '数量'}}
    }

series = [i for i in get_data_with_in('2015.12.24', '2016.01.05', ['北京二手手机','北京二手台式机/配件','北京二手笔记本'])]

charts.plot(series, options=options,show='inline')
# options=dict(title=dict(text='Charts are AWESOME!!!'))